# Code 1

In [40]:

import json
file_path = r"D:\yt-comment-sentiment-analysis2\experiment_info.json"
with open(file_path, 'r') as file:
    model_info = json.load(file)
model_info

{'run_id': '096d2a81a6114eba92a072921628a236', 'model_path': 'lgbm_model'}

In [32]:
import mlflow
from mlflow import MlflowClient
import os

os.environ['MLFLOW_TRACKING_USERNAME'] = "Pranay5519"
os.environ['MLFLOW_TRACKING_PASSWORD'] = os.getenv("DAGSHUB_PAT")

tracking_uri = "https://dagshub.com/Pranay5519/yt-comment-sentiment-analysis-2.mlflow"
client = MlflowClient(tracking_uri=tracking_uri)
model_uri = f"runs:/{model_info['run_id']}/{model_info['model_path']}"

model_name = "light_gbm_model"
try:
    client.get_registered_model(model_name)
    print(f"Model {model_name} already exists.")
except:
    print(f"Creating new registered model: {model_name}")
    client.create_registered_model(model_name)

Model light_gbm_model already exists.


In [33]:
versions = client.search_model_versions(f"name='{model_name}'")

for v in versions:
    print(f"Version: {v.version}, Stage: {v.current_stage}, Status: {v.status}")

Version: 4, Stage: None, Status: READY
Version: 3, Stage: None, Status: READY
Version: 2, Stage: None, Status: READY
Version: 1, Stage: None, Status: READY


In [34]:
model_info

{'run_id': '25beff545d4049fb8731de32644bdfd4', 'model_path': 'lgbm_model'}

In [41]:
from mlflow.tracking import MlflowClient

tracking_uri = "https://dagshub.com/Pranay5519/yt-comment-sentiment-analysis-2.mlflow"
client = MlflowClient(tracking_uri=tracking_uri)

model_name = "light_gbm_model"
run_id = model_info["run_id"]   # the run that logged the model

# 1. Find the model version created by this run
versions = client.search_model_versions(
    f"name='{model_name}'"
)

mv = None
for v in versions:
    if v.run_id == run_id:
        mv = v
        break

if mv is None:
    raise RuntimeError("No model version found for this run_id")

# 2. Set alias on THAT version
client.set_registered_model_alias(
    name=model_name,
    alias="staging",
    version=mv.version
)

print("Alias 'staging' set on version:", mv.version)


Alias 'staging' set on version: 5


In [39]:
mlflow.set_tracking_uri(
    "https://dagshub.com/Pranay5519/yt-comment-sentiment-analysis-2.mlflow"
)

model = mlflow.lightgbm.load_model("models:/light_gbm_model@staging")